# Proyecto 2 — Pipeline con **RFE** (ULTRA-RÁPIDO)
Versión optimizada para tiempo: **CV=2**, grids mínimos, y RFE con pocas opciones.
- Mantiene **RFE** en los 4 modelos y **SMOTE dentro de CV** (sin fugas).
- Pensado para terminar en ~10–15 minutos en CPU promedio.

**Entrada:** `creditcard.csv` en la misma carpeta.


## 1) Setup e importaciones

In [2]:

# !pip install -q scikit-learn imbalanced-learn pandas numpy
import os, numpy as np, pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve
RANDOM_STATE = 42
CV_FOLDS = 2


## 2) Utilidades

In [6]:

def cargar_datos(path_csv: str = "dataset/creditcard.csv") -> pd.DataFrame:
    df = pd.read_csv(path_csv)
    return df

def construir_preprocesador(feature_names):
    base_cols = ["Time", "Amount"]
    pca_cols = [c for c in feature_names if c.startswith("V")]
    return ColumnTransformer([
    ('all_features', StandardScaler(), base_cols + pca_cols)
])
def metricas_desde_confusion(cm):
    tn, fp, fn, tp = cm.ravel()
    acc = (tp + tn) / (tp + tn + fp + fn + 1e-12)
    sensibilidad = tp / (tp + fn + 1e-12)
    especificidad = tn / (tn + fp + 1e-12)
    precision = tp / (tp + fp + 1e-12)
    f1 = 2 * precision * sensibilidad / (precision + sensibilidad + 1e-12)
    return dict(exactitud=acc, sensibilidad=sensibilidad, especificidad=especificidad,
                precision=precision, f1=f1, tp=int(tp), fp=int(fp), tn=int(tn), fn=int(fn))

def evaluar_modelo(nombre, y_true, y_pred, y_score=None):
    print(f"\\n=== {nombre} ===")
    print(classification_report(y_true, y_pred, digits=4))
    cm = confusion_matrix(y_true, y_pred)
    print("Matriz de confusión:\\n", cm)
    resumen = metricas_desde_confusion(cm)
    if y_score is not None:
        try:
            auc = roc_auc_score(y_true, y_score)
            resumen["roc_auc"] = auc
            print(f"ROC-AUC: {auc:.4f}")
        except Exception as e:
            print("No fue posible calcular ROC-AUC:", e)
    return resumen


## 3) Pipelines con RFE (grids mínimos, CV=2)

In [4]:

from sklearn.pipeline import Pipeline

def pipeline_knn_rfe(feature_names):
    pre = construir_preprocesador(feature_names)
    rfe_est = LogisticRegression(solver="liblinear", max_iter=200, random_state=RANDOM_STATE)
    rfe = RFE(estimator=rfe_est, n_features_to_select=3, step=2)
    knn = KNeighborsClassifier()
    pipe = ImbPipeline([("pre", pre), ("smote", SMOTE(random_state=RANDOM_STATE, k_neighbors=5)), ("rfe", rfe), ("clf", knn)])
    param_grid = {"rfe__n_features_to_select": [16, 20], "clf__n_neighbors": [5], "clf__metric": ["manhattan", "euclidean"]}
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    return GridSearchCV(pipe, param_grid, scoring="f1", cv=cv, n_jobs=-1, refit=True, verbose=1)

def pipeline_arbol_rfe(feature_names):
    pre = construir_preprocesador(feature_names)
    rfe = RFE(estimator=DecisionTreeClassifier(random_state=RANDOM_STATE), n_features_to_select=3, step=2)
    dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
    pipe = ImbPipeline([("pre", pre), ("smote", SMOTE(random_state=RANDOM_STATE, k_neighbors=5)), ("rfe", rfe), ("clf", dt)])
    param_grid = {"rfe__n_features_to_select": [16, 20], "clf__max_depth": [15, None], "clf__min_samples_leaf": [1, 2]}
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    return GridSearchCV(pipe, param_grid, scoring="f1", cv=cv, n_jobs=-1, refit=True, verbose=1)

def pipeline_svm_rfe(feature_names):
    pre = construir_preprocesador(feature_names)
    rfe = RFE(estimator=LinearSVC(random_state=RANDOM_STATE, tol=1e-3, C=1.0), n_features_to_select=3, step=2)
    svm = LinearSVC(random_state=RANDOM_STATE)
    pipe = ImbPipeline([("pre", pre), ("smote", SMOTE(random_state=RANDOM_STATE, k_neighbors=5)), ("rfe", rfe), ("clf", svm)])
    param_grid = {"rfe__n_features_to_select": [16, 20], "clf__C": [1], "clf__tol": [1e-3, 1e-4]}
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    return GridSearchCV(pipe, param_grid, scoring="f1", cv=cv, n_jobs=-1, refit=True, verbose=1)

def pipeline_rf_rfe(feature_names):
    pre = construir_preprocesador(feature_names)
    rfe = RFE(estimator=RandomForestClassifier(n_estimators=150, random_state=RANDOM_STATE), n_features_to_select=3, step=2)
    rf = RandomForestClassifier(random_state=RANDOM_STATE)
    pipe = ImbPipeline([("pre", pre), ("smote", SMOTE(random_state=RANDOM_STATE, k_neighbors=5)), ("rfe", rfe), ("clf", rf)])
    param_grid = {"rfe__n_features_to_select": [16, 20], "clf__n_estimators": [200], "clf__max_depth": [None, 15], "clf__min_samples_leaf": [1, 2]}
    cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    return GridSearchCV(pipe, param_grid, scoring="f1", cv=cv, n_jobs=-1, refit=True, verbose=1)


## 4) Carga y split 70/30

In [7]:

# === Cargar datos y split ===
path_csv = "dataset/creditcard.csv"
if not os.path.exists(path_csv):
    raise FileNotFoundError("No se encontró creditcard.csv (Kaggle: mlg-ulb/creditcardfraud).")
df = cargar_datos(path_csv)
feature_cols = ["Time", "Amount"] + [f"V{i}" for i in range(1, 29)]
X = df[feature_cols].copy()
y = df["Class"].astype(int).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=RANDOM_STATE)
print("Información del dataset:")
print(f"Dimensiones: {df.shape}")
print(f"Distribución de clases: {df['Class'].value_counts()}")
print(f"Porcentaje de fraude: {df['Class'].mean()*100:.4f}%")
print(f"Valores nulos: {df.isnull().sum().sum()}")
print(f"Fraude total: {y.mean()*100:.4f}% | Train: {y_train.mean()*100:.4f}% | Test: {y_test.mean()*100:.4f}%")


Información del dataset:
Dimensiones: (284807, 31)
Distribución de clases: Class
0    284315
1       492
Name: count, dtype: int64
Porcentaje de fraude: 0.1727%
Valores nulos: 0
Fraude total: 0.1727% | Train: 0.1725% | Test: 0.1732%


## 5) Entrenamiento + evaluación + resumen

In [11]:
def extraer_caracteristicas(grid, nombre_modelo):
    best_pipe = grid.best_estimator_
    rfe = best_pipe.named_steps["rfe"]
    seleccionadas = [f for f, s in zip(feature_cols, rfe.support_) if s]
    print(f"\n {nombre_modelo}:")
    print("Características seleccionadas:", seleccionadas)
    return seleccionadas

In [10]:

# === Entrenamiento + GridSearchCV (RFE ULTRA-RÁPIDO) ===
modelos = {
    "k-NN (RFE+LR)": pipeline_knn_rfe(feature_cols),
    "Árbol (RFE+Árbol)": pipeline_arbol_rfe(feature_cols),
    "SVM lineal (RFE+LinearSVC)": pipeline_svm_rfe(feature_cols),
    "Random Forest (RFE+RF)": pipeline_rf_rfe(feature_cols),
}
resultados = {}
caracteristicas_seleccionadas = {}

for nombre, grid in modelos.items():
    print("\n" + "="*80)
    print(f"Entrenando {nombre} ...")
    grid.fit(X_train, y_train)
    print(f"Mejores hiperparámetros ({nombre}): {grid.best_params_}")
    best = grid.best_estimator_
    
    rfe_step = best.named_steps["rfe"]
    mask = rfe_step.support_
    selected_features = [f for f, s in zip(feature_cols, mask) if s]
    caracteristicas_seleccionadas[nombre] = selected_features
    
    y_pred = best.predict(X_test)
    y_score = None
    if hasattr(best, "decision_function"):
        try:
            y_score = best.decision_function(X_test)
        except Exception:
            y_score = None
    if y_score is None and hasattr(best, "predict_proba"):
        try:
            y_score = best.predict_proba(X_test)[:, 1]
        except Exception:
            y_score = None
    resumen = evaluar_modelo(nombre, y_test, y_pred, y_score)
    resultados[nombre] = {"mejores_hiperparametros": grid.best_params_, **resumen}

import pandas as pd
resumen_df = pd.DataFrame(resultados).T[["sensibilidad","especificidad","precision","f1","exactitud"]]\
             .sort_values(by="sensibilidad", ascending=False)
display(resumen_df)
resumen_df.to_csv("resumen_metricas_modelos_RFE_ultrafast.csv", index=True)
print("CSV guardado: resumen_metricas_modelos_RFE_ultrafast.csv")



Entrenando k-NN (RFE+LR) ...
Fitting 2 folds for each of 4 candidates, totalling 8 fits
Mejores hiperparámetros (k-NN (RFE+LR)): {'clf__metric': 'manhattan', 'clf__n_neighbors': 5, 'rfe__n_features_to_select': 20}
\n=== k-NN (RFE+LR) ===
              precision    recall  f1-score   support

           0     0.9997    0.9978    0.9988     85295
           1     0.4026    0.8378    0.5439       148

    accuracy                         0.9976     85443
   macro avg     0.7012    0.9178    0.7713     85443
weighted avg     0.9987    0.9976    0.9980     85443

Matriz de confusión:\n [[85111   184]
 [   24   124]]
ROC-AUC: 0.9217

Entrenando Árbol (RFE+Árbol) ...
Fitting 2 folds for each of 8 candidates, totalling 16 fits
Mejores hiperparámetros (Árbol (RFE+Árbol)): {'clf__max_depth': None, 'clf__min_samples_leaf': 2, 'rfe__n_features_to_select': 20}
\n=== Árbol (RFE+Árbol) ===
              precision    recall  f1-score   support

           0     0.9995    0.9977    0.9986     85295
  

,sensibilidad,especificidad,precision,f1,exactitud
SVM lineal (RFE+LinearSVC),0.871622,0.981359,0.075044,0.13819,0.981169
k-NN (RFE+LR),0.837838,0.997843,0.402597,0.54386,0.997566
Random Forest (RFE+RF),0.790541,0.999695,0.818182,0.804124,0.999333
Árbol (RFE+Árbol),0.695946,0.997749,0.349153,0.465011,0.997226


CSV guardado: resumen_metricas_modelos_RFE_ultrafast.csv


In [ ]:
caracteristicas_seleccionadas

## 6) (Opcional) Ajuste de umbral

In [9]:

# === (Opcional) Ajuste de umbral sobre el modelo con mayor sensibilidad ===
mejor_modelo = resumen_df.index[0]
best_estimator = modelos[mejor_modelo].best_estimator_
if hasattr(best_estimator, "predict_proba") or hasattr(best_estimator, "decision_function"):
    scores = best_estimator.predict_proba(X_test)[:, 1] if hasattr(best_estimator, "predict_proba") else best_estimator.decision_function(X_test)
    precisions, recalls, thresholds = precision_recall_curve(y_test, scores)
    objetivo = 0.90
    import numpy as np
    idx = np.where(recalls >= objetivo)[0]
    if len(idx) > 0:
        thr = thresholds[idx[0]-1] if idx[0] > 0 else thresholds[0]
        y_pred_thr = (scores >= thr).astype(int)
        print(f"Ajuste de umbral a ~{objetivo:.2f} de sensibilidad — threshold={thr:.4f}")
        _ = evaluar_modelo(f"{mejor_modelo} (umbral ajustado)", y_test, y_pred_thr, scores)
    else:
        print("No se logró alcanzar la sensibilidad objetivo en test.")
else:
    print("El mejor modelo no entrega score continuo; se omite ajuste de umbral.")


Ajuste de umbral a ~0.90 de sensibilidad — threshold=-8.2775
\n=== SVM lineal (RFE+LinearSVC) (umbral ajustado) ===
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000     85295
           1     0.0017    1.0000    0.0035       148

    accuracy                         0.0017     85443
   macro avg     0.0009    0.5000    0.0017     85443
weighted avg     0.0000    0.0017    0.0000     85443

Matriz de confusión:\n [[    0 85295]
 [    0   148]]
ROC-AUC: 0.9663


c:\Users\none\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\none\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\none\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag